## **Building a Text Translation Application**

Here, we will be using LCEL (Langchain Expression language) to build a simple text translation application using some prompt engineering and LLM calls.

After this tutorial, you will have a high level knowledge on:
- Using Language Models
- Using Langchain Expression Language (LCEL) to chain components together.
- Using Prompt Templates and Output Parsers
- Deploying your application with Langserve

**Let's Dive In!**

## Setup

### Installation

**Note:** If this is your first time using jupyter notebook, follow the link [here](https://jupyter.org/install) on how to set it up

In [1]:
! pip install langchain

  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none

## Using Language Models

We will start by learning how to use a language model by itself. Here, we will be using `llama3.1` by Meta and we will be accessing it via `groq` API.

**Note:** To use groq, you need an API key and here is how you can get one

In [2]:
! pip install -qU langchain-groq

In [4]:
# This will be used for loading our environment variables - like the API keys
! pip install python-dotenv

**Load Environment Variables**

In [5]:
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

**Using the Groq API**

In [7]:
from langchain_groq import ChatGroq

In [8]:
# Create an instance of the llama-3.1-8b-instant model

model = ChatGroq(model="llama-3.1-8b-instant")

Langchain `ChatModels` (e.g ChatGroq) are instances of Langchain "Runnables" and all runnables have a standard way of interacting them.

To call the `model` object created above, we simply need to pass a list of messages (HumanMessage and SystemMessage) to the `.invoke` method.

- HumanMessage: These are messgaes that are passed in from a human to the model.
- SystemMessage: This message is used for controllling the behavior of the AI model.

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

In [15]:
messages = [
    SystemMessage(content="Translate the following from English to Italian and only return the translation."),
    HumanMessage(content="hello")
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 29, 'total_tokens': 33, 'completion_time': 0.005333333, 'prompt_time': 0.007422644, 'queue_time': None, 'total_time': 0.012755977}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-74d75d48-a0d8-48bf-a434-fc9b8fa69e4d-0', usage_metadata={'input_tokens': 29, 'output_tokens': 4, 'total_tokens': 33})

## Output Parsers

As noticed, the response from the model contains an object "AIMessage", whcih contains a string response (content) and other metadata.

Most times, we just want to work with the string and here is how we get the model to return the string using an output parser.

In [16]:
from langchain_core.output_parsers import StrOutputParser

In [17]:
parser = StrOutputParser()

One way to use the output parser is to call it as it is:

In [18]:
response = model.invoke(messages)

In [19]:
parser.invoke(response)

'Ciao'

The better way to use the output parser is to chain it with the model. This way, the parser gets called authomatically everytime the model is called.

To chain the parser with the model, we will user the `|` operator. The `|` operator is used to chain multiple components together in langchain.

In [20]:
chain = model | parser

In [21]:
chain.invoke(messages)

'Ciao!'

## Prompt Template

Instead of hard coding the user and system messages, prompt template allows for the flexibility to add variables to the system message and get dynamic inputs from the user.


Then, the PromptTemplate takes these inputs and returns data (prompt) that is ready to be used by the model.

Let's create a PromptTemplate here. It will take in two user variables:

- language: The language to translate text into
- text: The text to translate

In [22]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that will be formated into the system message

In [23]:
system_template = "Translate the following into {language}:"

Then create the prompt template, which will be a combination of the `system_template` and a simple template containing text input to be translated from the user.

In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system",system_template), ("user", "{text}")]
)

The input to the template is a dictionary. Let's try it out

In [25]:
result = prompt_template.invoke({"language":"Italian", "text":"Hello"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Italian:'), HumanMessage(content='Hello')])

## Chaining the Components Together using LCEL

We can now combine the `prompt_template`, the `model` and `output_parser` to form a single chain.

In [26]:
chain = prompt_template | model | parser

In [27]:
chain.invoke({"language":"Italian", "text":"Hello"})

'Ciao'

## Deploying the Application

Here, we will be making use of Langserve, which helps developers deploy Langchain applications as a REST API.

### Creating the Server

To create a server for our application we'll make two file `app.py` and `serve.py` file. The `serve.py` will contain our logic for serving our application. It consists of three things:

1. The call to the chain that we just built above (simple import)
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with langserve.add_routes

The `app.py` file will consist of the definition of our chain - exactly what we have done above